# Loading Data

In [1]:
from data import load_data
import numpy as np

clinical, _, genes, treatments, outcome = load_data()

print(clinical.shape)

clinical.describe().T

(724, 35)


,count,mean,std,min,25%,50%,75%,max
cmmc,376.0,5907.877660,9307.921203,0.00000,233.25000,1409.50000,7145.500000,47146.00000
ecog_ps,357.0,1.330532,0.638197,1.00000,1.00000,1.00000,1.000000,4.00000
percent_aneuploid,539.0,18.178948,21.639551,0.00000,0.00000,10.00000,29.700000,89.40000
percent_plama_cells_bone_marrow,616.0,17.599838,16.658980,0.00000,5.70000,11.60000,23.225000,84.30000
percent_plama_cells_peripherical_blood,616.0,0.636519,2.831528,0.00000,0.00000,0.00000,0.100000,33.30000
creatinine,710.0,106.869476,62.670669,33.00000,70.72000,88.40000,116.688000,503.88000
iss,704.0,1.917614,0.799379,1.00000,1.00000,2.00000,3.000000,3.00000
absolute_neutrophil,707.0,3.912428,2.173153,0.58000,2.40000,3.50000,4.760000,16.51200
platelet,723.0,220.117566,79.182326,18.00000,167.00000,215.00000,262.000000,668.00000
wbc_x10_10_9_l,616.0,6.271688,2.517359,1.40000,4.50000,5.90000,7.500000,25.80000


# Gene Expressions

In [2]:
genes.iloc[:8, :8]

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036
ID,,,,,,,,
MMRF1021,959,0,1264,668,244,27,3711,1016
MMRF1024,776,0,972,595,78,504,10,1263
MMRF1029,1470,0,2143,1093,209,36,20,3677
MMRF1030,83,0,1235,422,58,42,21,1714
MMRF1031,2,0,1127,432,190,48,117,2527
MMRF1032,24,0,748,214,62,33,65,571
MMRF1033,18,0,827,478,46,211,5,962
MMRF1037,18,0,870,385,66,10,22,1165


# Treatments

In [3]:
treatments.head()

,therapy_first_line_Bor-Cyc-Dex,therapy_first_line_Bor-Dex,therapy_first_line_Bor-Len-Dex,therapy_first_line_Len-Dex,therapy_first_line_Non-therapy
ID,,,,,
MMRF1021,0,0,1,0,0
MMRF1024,0,0,0,0,1
MMRF1029,0,0,1,0,0
MMRF1030,0,0,1,0,0
MMRF1031,0,0,1,0,0


# THERAPY SENSITIVITY MODELLING

In [4]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

encoder = LabelEncoder()

te = encoder.fit_transform(treatments.idxmax(axis=1))

te = pd.DataFrame({'treatment': te}, index=treatments.index)

te.head()

,treatment
ID,
MMRF1021,2
MMRF1024,4
MMRF1029,2
MMRF1030,2
MMRF1031,2


## Cross-validation Experiment

In [8]:
from pipeline import MuLT

from sklearn.model_selection import StratifiedKFold
from evaluation import optimize_threshold, classification_metrics
from sklearn.metrics import roc_auc_score, log_loss, confusion_matrix


from constants import N_FOLDS, RANDOM_STATE
from util import join_values

import lightgbm as lgb
import pickle as pkl
import pandas as pd
import numpy as np
import time
import os

# creating analyser object to compute and group 
# classification matrics grouped by training and validation
# dataset and by experiment id
# analyser = Analyser()

# Creating 10-fold CV splits stratified by treatment and outcome
kfold = StratifiedKFold(N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
split = kfold.split(np.zeros(outcome.shape[0]), join_values([treatments, outcome]))

#
#
result = {c: [] for c in ['experiment', 'train_auc', 'valid_auc', 
                          'train_loss', 'valid_loss', 'execution_time', 'threshold']}

for experiment, (train_index, valid_index) in enumerate(split):
    
    initial_time = time.time()
    
    print('{}\n\n'.format(experiment))
    
    #######################################################################################################
    # Split train & valid
    #######################################################################################################
    
    response_train = outcome.iloc[train_index, 0]
    response_valid = outcome.iloc[valid_index, 0]

    clinical_train = clinical.iloc[train_index, :]
    clinical_valid = clinical.iloc[valid_index, :]

    treatments_train = te.iloc[train_index, :]
    treatments_valid = te.iloc[valid_index, :]
    
    genes_train = genes.iloc[train_index, :]
    genes_valid = genes.iloc[valid_index, :]
    
    #######################################################################################################
    # Train & Test distances
    #######################################################################################################

    if False:
        
        dists = []

        for row_train in clinical.join(genes, how='inner').values:
            for row_valid in clinical.join(genes, how='inner').values:
                dists.append(np.linalg.norm(row_train-row_valid))

        train_test_distance_avg = np.mean(dists)
        train_test_distance_std = np.std(dists)
        train_test_distance_min = np.min(dists)
        train_test_distance_max = np.max(dists)

    #######################################################################################################
    # MuLT fitting
    #######################################################################################################
    
    mult = MuLT(experiment_number=experiment, 
                number_of_experiments=N_FOLDS, 
                output_path='./output/mult/', 
                random_state=RANDOM_STATE)

    mult.fit(clinical_train, genes_train, treatments_train, response_train, 

        lgb_fixed_parameters = {
            'metric': 'binary_logloss',
            'n_estimators': 100,
            'objective': 'binary',
            'is_unbalance': False, 
            'extra_trees': True},

        optimization_n_call=100,
        optimization_n_folds=2,
        optimization_early_stopping_rounds=1,

        clinical_marker_selection_threshold=0.05,
        gene_selection_threshold=0.0005,

        dae_decay_rate=1.0,
        dae_learning_rate=1e-1,
        dae_steps=100000,
        dae_early_stopping_rounds=1000,

        lgb_early_stopping_rounds=1,

        predictor_n_folds=3)

    with open('output/mult/trained_model_{}.pkl'.format(experiment), 'wb') as file:
        pkl.dump(mult, file)
    
    #######################################################################################################
    # MuLT inference
    #######################################################################################################

    y_hat_train = mult.predict(clinical_train, genes_train, treatments_train)
    y_hat_valid = mult.predict(clinical_valid, genes_valid, treatments_valid)

    #################################################################################################
    # Analysing Performance
    #################################################################################################   

    # Computing AUC
    train_auc = roc_auc_score(response_train, y_hat_train)
    valid_auc = roc_auc_score(response_valid, y_hat_valid)

    # Computing logLoss
    train_loss = log_loss(response_train, y_hat_train)
    valid_loss = log_loss(response_valid, y_hat_valid)

    # Compute optimized threshold
    opt_threshold = optimize_threshold(response_train, y_hat_train)

    if opt_threshold is None:
        opt_threshold = np.mean(response_train)

    # compute confusion matrix
    tn, fp, fn, tp = confusion_matrix(response_valid, [int(y >= opt_threshold) for y in y_hat_valid]).ravel()

    classification_results = classification_metrics(tn, fp, fn, tp)

    # add results to data frame (dict for now)
    for k in classification_results:
        if k not in result:
            result[k] = []
        result[k].append(classification_results[k])

    result['experiment'].append(experiment)
    result['train_auc'].append(train_auc)
    result['valid_auc'].append(valid_auc)
    result['train_loss'].append(train_loss)
    result['valid_loss'].append(valid_loss)
    result['execution_time'].append(time.time() - initial_time)
    result['threshold'].append(opt_threshold)

    print('Experiment {} with {} genes and {} clinical markers'.format(
          experiment, len(mult.selected_genes[0]), len(mult.selected_clinical[0])))

    print('Train: {}'.format(train_auc))

    print('Valid: {}'.format(valid_auc))

    print("\n========================================================================================\n")
    
    # Exporting inference
    response_train = pd.DataFrame(response_train)
    response_train['y_hat'] = y_hat_train
    response_train.to_csv('output/mult/inference/train_{}.csv'.format(experiment), index=True, sep=',')
    
    response_valid = pd.DataFrame(response_valid)
    response_valid['y_hat'] = y_hat_valid
    response_valid.to_csv('output/mult/inference/valid_{}.csv'.format(experiment), index=True, sep=',')

0


early stopping after 1000 iterations without improvements with 1868 steps: best metric value 130.25230407714844
INFO:tensorflow:Restoring parameters from C:\Users\Venezian\git\multiple-myeloma\output\mult\dae\data_augmentation_adadelta_000\graph\data_augmentation_adadelta_000
INFO:tensorflow:Restoring parameters from C:\Users\Venezian\git\multiple-myeloma\output\mult\dae\data_augmentation_adadelta_000\graph\data_augmentation_adadelta_000
TRAIN mean log loss: 0.5646441245084735
TRAIN mean AUC: 0.835172941536578
VALID mean log loss: 0.5637330419210866
VALID mean AUC: 0.7900397918579737
INFO:tensorflow:Restoring parameters from C:\Users\Venezian\git\multiple-myeloma\output\mult\dae\data_augmentation_adadelta_000\graph\data_augmentation_adadelta_000
INFO:tensorflow:Restoring parameters from C:\Users\Venezian\git\multiple-myeloma\output\mult\dae\data_augmentation_adadelta_000\graph\data_augmentation_adadelta_000
Experiment 0 with 16 genes and 6 clinical markers
Train: 0.7186016004197823

In [ ]:
result = pd.DataFrame(result)

result.to_csv('output/mult/metrics.csv', sep=',', index=False)

result

In [ ]:
result.mean()

In [ ]:
result.std()

## Analyse per Treatment

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

def classification_metrics(x):
    try:
        
        tn, fp, fn, tp = confusion_matrix(x['y_true'], x['y_hat_hard']).ravel()
        
        sensitivity, specificity = tp / (tp + fn), tn / (tn + fp)
        
        precision = tp / (tp + fp)
        
        return pd.Series({
            'AUC': roc_auc_score(x['y_true'], x['y_hat_adjusted']),
            'Accuracy': accuracy_score(x['y_true'], x['y_hat_hard']),
            'Sensitivity': sensitivity,
            'Specificity': specificity,
            'Precision': precision,
            'F1': 2 * precision * sensitivity / (precision + sensitivity)})
    
    except:
        return pd.Series({
            'AUC': 0.0,
            'Accuracy': 0.0,
            'Sensitivity': 0.0,
            'Specificity': 0.0,
            'Precision': 0.0,
            'F1': 0.0})
    
BASE_DIR = 'output/mult/inference/'
    
df = None

metrics = pd.read_csv('output/mult/metrics.csv', sep=',')

for file in os.listdir(BASE_DIR):
    if 'valid' in file:
        index = file.split('_')[1].split('.')[0]
        train = pd.read_csv(os.path.join(BASE_DIR, 'train_{}.csv'.format(index)), sep=',')
        t = metrics[metrics['experiment'] == int(index)]['threshold'].values[0]
        tmp = pd.read_csv(os.path.join(BASE_DIR, file), sep=',')
        df = tmp if df is None else pd.concat([df, tmp], axis=0)
        df['y_hat_hard'] = df['y_hat'].apply(lambda x: 1 if x >= t else 0)
        
df = df.set_index('ID', drop=True)

df = df.join(treatments[treatments==1].stack().reset_index().drop(0,1).set_index('ID').rename(columns={'level_1': 'treatment'}))

df = df.rename(columns={'response_best_response_first_line': 'y_true'})

df.groupby('treatment').apply(classification_metrics)

In [ ]:
df.groupby('treatment')['y_true'].count()